In [ ]:
import requests, json

In [ ]:
sensors = {
    "sonometer": "cVDqRI0BmK9pjniFeoOV",
    "depth": "jRCJ548B1ljoqTiFLu0b",
    "weather": "5rTM-4sB-iVpGSRJcSJV",
    "counter_indoor": "hup4zYsBoTGoLusGWiIj",
    "satellite": "2DjL-4sBzJIAy7teICbv",
    "buoy": "UVztVI0BaNT-uedBhHl1"
}

def get_url(sensor_id):
    return f"https://platon.grc.upv.es/sensingtools-api/api/user-hard-sensors/{sensor_id}/data/latest-values"

In [ ]:
for sensor, sensor_id in sensors.items():
    response = requests.get(get_url(sensor_id), verify=False)
    with open(f"sensors-data/{sensor}.json", 'w') as f:
        json.dump(response.json(), f, indent=4)

In [ ]:
for d in data['data']:
    print(d['sensor_type'])

In [ ]:
response = requests.get("http://localhost:3000/data").json()

ids = set()
for d in response['data']:
    ids.add(d['location_id'])
len(ids)

In [ ]:
from dataclasses import dataclass

@dataclass(frozen=True)
class SensorInfo:
    location_id: str # <sensorId>_<name>_<lat>_<lon>_<alt>
    safe_location_id: str
    sensor_id: str
    name: str
    timestamp: int # Unix ms timestamp
    lat: str
    lon: str
    alt: str

    def __eq__(self, value: object) -> bool:
        return isinstance(value, SensorInfo) and value.location_id == self.location_id
    
    def __hash__(self) -> int:
        return hash(self.location_id)

class SensorMeasurement:
    def __init__(self, sensor_info: SensorInfo, data: dict):
        self.sensor_info = sensor_info
        for key, value in data.items():
            self.__setattr__(key, value)
    
    def toJSON(self):
        return json.dumps(self, default=lambda o: o.__dict__, indent=4)

In [ ]:
from base64 import b64encode

def parse_request_data(sensor_id, data) -> dict[SensorInfo, dict[str, str]]:
    sensors_data: dict[SensorInfo, dict[str, str]] = {}
    for data in response['data']:
        sensor_info = SensorInfo(
            location_id=data['location_id'],
            safe_location_id=b64encode(data['location_id'].encode()),
            sensor_id=sensor_id,
            name=data['name'],
            timestamp=data['timestamp'],
            lat=data['lat'],
            lon=data['lon'],
            alt=data['alt']
        )
        if sensor_info not in sensors_data:
            sensors_data[sensor_info] = {}
        sensors_data[sensor_info][data['sensor_type']] = data['value']
    return sensors_data

In [ ]:
endpoints = ["buoy", "counter_indoor", "depth", "satellite", "sonometer", "weather"]
inputs = []
result = []

for endpoint in endpoints:
    response = requests.get(f"http://localhost:3000/{endpoint}").json()
    inputs.append(response)
    sensors_data = parse_request_data(sensors[endpoint], response)
    result.append(sensors_data)

In [ ]:
from paho.mqtt import client as mqtt_client

broker = 'localhost'
port = 1883
# topic = "sensors/"

def connect_mqtt() -> mqtt_client.Client:
    def on_connect(client, userdata, flags, rc):
        if rc == 0:
            print("Connected to MQTT Broker!")
        else:
            print("Failed to connect, return code %d\n", rc)

    client = mqtt_client.Client()
    client.on_connect = on_connect
    client.connect(broker, port)
    return client

def publish(client, topic, msg):
    result = client.publish(topic, msg)
    status = result[0]
    if status == 0:
        print(f"Send `{msg}` to topic `{topic}`")
    else:
        print(f"Failed to send message to topic {topic}")

client = connect_mqtt()
# client.loop_start()

for data in result:
    for sensor_info, data in data.items():
        sensor_measurement = SensorMeasurement(sensor_info, data)
        publish(client, f"sensors/{sensor_measurement.sensor_info.location_id}", sensor_measurement.toJSON())

In [ ]:
measure = SensorMeasurement(sensor_info, sensors_data[sensor_info])

for sensor_info, data in sensors_data.items():
    measure = SensorMeasurement(sensor_info, data)
    print(measure.toJSON())